# Softmax fast approximations

Короче, выбрал этот странный mxnet т.к. там 1)очень простая предобработка (причем она есть для всех датасетов, стоит только поменять dataset_name) и 2)есть предобученные языковые модели на этих датасетах, так что можно не тратить время на обучение вообще - никаких gpu не надо

По сути тут все операции как в numpy (вместо np пиши nd и все)

Про gluonnlp & mxnet за 5 минут https://beta.mxnet.io/guide/crash-course/index.html

Пока есть только подсчет времени работы произвольной сети (в evaluate)

Надо:

1. написать таки SVD Softmax слой
2. **(contribute here, please)** hierarchical softmax layer (как минимум) ну или чего еще из related work у нас есть - если видите что просто реализовывается - попробуйте
3. графички зависимости от W времени и качества (перплексии) для SVD-softmax
4. кто-то будет презентовать это в четверг (почти наверное я не смогу)


* текущий код работает на cpu, на gpu тоже работал, но я сейчас это снес, если надо будет (зачем?) - напишите мне
* 1 & 3 пункты я по-любому сам доделаю в среду
* код ниже в принципе не важен (вам его вряд ли стоит запускать) - все что требуется - напишите слои для аппроксимации softmax

In [1]:
import warnings
warnings.filterwarnings('ignore')

import glob
import time
import math

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.utils import download
from mxnet import ndarray as nd

import gluonnlp as nlp
import numpy as np

In [2]:
num_gpus = 0
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else mx.cpu()
log_interval = 200

In [3]:
batch_size = 20 #* len(context)
lr = 20
epochs = 3
bptt = 35
grad_clip = 0.25

In [4]:
dataset_name = 'wikitext-2'
train_dataset, val_dataset, test_dataset = [
    nlp.data.WikiText2(
        segment=segment, bos=None, eos='<eos>', skip_empty=False)
    for segment in ['train', 'val', 'test']
]

vocab = nlp.Vocab(
    nlp.data.Counter(train_dataset), padding_token=None, bos_token=None)

bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
    vocab, bptt, batch_size, last_batch='discard')
train_data, val_data, test_data = [
    bptt_batchify(x) for x in [train_dataset, val_dataset, test_dataset]
]

In [5]:
model_name = 'standard_lstm_lm_200'
model, vocab = nlp.model.get_model(model_name, vocab=vocab, 
                                   dataset_name=dataset_name, pretrained=True, 
                                   ctx=context)
print(model)
print(vocab)

# model.initialize(mx.init.Xavier(), ctx=context)

trainer = gluon.Trainer(model.collect_params(), 'sgd', {
    'learning_rate': lr,
    'momentum': 0,
    'wd': 0
})
loss = gluon.loss.SoftmaxCrossEntropyLoss()

StandardRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 200, float32)
    (1): Dropout(p = 0.2, axes=())
  )
  (encoder): LSTM(200 -> 200, TNC, num_layers=2, dropout=0.2)
  (decoder): HybridSequential(
    (0): Dense(200 -> 33278, linear)
  )
)
Vocab(size=33278, unk="<unk>", reserved="['<eos>']")


In [6]:
params = model.collect_params()
params

standardrnn0_ (
  Parameter standardrnn0_hybridsequential0_embedding0_weight (shape=(33278, 200), dtype=float32)
  Parameter standardrnn0_lstm0_l0_i2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_h2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_i2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_h2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_i2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_h2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_i2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_h2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_hybridsequential0_embedding0_bias (shape=(33278,), dtype=float32)
)

In [7]:
W = params['standardrnn0_hybridsequential0_embedding0_weight'].data().asnumpy()
b = params['standardrnn0_hybridsequential0_embedding0_bias'].data().asnumpy()
W.shape, b.shape

((33278, 200), (33278,))

In [8]:
%%time
U, Sigma, V = np.linalg.svd(W, full_matrices=False)

CPU times: user 835 ms, sys: 100 ms, total: 935 ms
Wall time: 521 ms


In [9]:
U.shape, Sigma.shape, V.shape

((33278, 200), (200,), (200, 200))

In [10]:
def detach(hidden):
    if isinstance(hidden, (tuple, list)):
        hidden = [detach(i) for i in hidden]
    else:
        hidden = hidden.detach()
    return hidden

def evaluate(model, data_source, batch_size, ctx):
    total_L = 0.0
    total_time = 0.0
    ntotal = 0
    hidden = model.begin_state(
        batch_size=batch_size, func=mx.nd.zeros, ctx=ctx)
    for i, (data, target) in enumerate(data_source):
        data = data.as_in_context(ctx)
        target = target.as_in_context(ctx)
        t_begin = time.time()
        output, hidden = model(data, hidden)
        hidden = detach(hidden)
        L = loss(output.reshape(-3, -1), target.reshape(-1))
        total_L += mx.nd.sum(L).asscalar()
        single_t = time.time() - t_begin
        print(f'batch {i} inference time: {single_t:.7f}')
        total_time += single_t
        ntotal += target.reshape(-1).size
    return total_L / ntotal, total_time / ntotal

In [12]:
evaluate(model, test_data, batch_size, context)

batch 0 inference time: 0.3381550
batch 1 inference time: 0.2853339
batch 2 inference time: 0.2853391
batch 3 inference time: 0.2871799
batch 4 inference time: 0.2963479
batch 5 inference time: 0.3045039
batch 6 inference time: 0.2889631
batch 7 inference time: 0.3018198
batch 8 inference time: 0.3025203
batch 9 inference time: 0.2889941
batch 10 inference time: 0.2942410
batch 11 inference time: 0.2948592
batch 12 inference time: 0.2909350
batch 13 inference time: 0.4573598
batch 14 inference time: 0.2910721
batch 15 inference time: 0.3886399
batch 16 inference time: 0.2938199
batch 17 inference time: 0.2966020
batch 18 inference time: 0.2973442
batch 19 inference time: 0.2957962
batch 20 inference time: 0.2886200
batch 21 inference time: 0.2932250
batch 22 inference time: 0.2909870
batch 23 inference time: 0.2901282
batch 24 inference time: 0.2916729
batch 25 inference time: 0.2870910
batch 26 inference time: 0.3891699
batch 27 inference time: 0.2976661
batch 28 inference time: 0.301

batch 231 inference time: 0.3034580
batch 232 inference time: 0.3208168
batch 233 inference time: 0.2967632
batch 234 inference time: 0.3108890
batch 235 inference time: 0.3079271
batch 236 inference time: 0.3250930
batch 237 inference time: 0.3111029
batch 238 inference time: 0.2859759
batch 239 inference time: 0.3233099
batch 240 inference time: 0.3105931
batch 241 inference time: 0.3215377
batch 242 inference time: 0.3195000
batch 243 inference time: 0.2983119
batch 244 inference time: 0.3450181
batch 245 inference time: 0.3218057
batch 246 inference time: 0.2971511
batch 247 inference time: 0.2958679
batch 248 inference time: 0.2833960
batch 249 inference time: 0.2972338
batch 250 inference time: 0.2889900
batch 251 inference time: 0.2973111
batch 252 inference time: 0.2942610
batch 253 inference time: 0.2892818
batch 254 inference time: 0.3495562
batch 255 inference time: 0.3321617
batch 256 inference time: 0.2970431
batch 257 inference time: 0.2852890
batch 258 inference time: 0.

(4.622447090242347, 0.00044113784322933274)

In [12]:
class Softmax(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super(Softmax, self).__init__(**kwargs)

    def hybrid_forward(self, F, x):
        return F.softmax(x)
    
class SVDSoftmax(gluon.HybridBlock):
    # TODO
    def __init__(self, svd_decomposition, bias, **kwargs):
        super(Softmax, self).__init__(**kwargs)
        U, Sigma, V_t = svd_decomposition
        self.B = np.dot(U, nd.diag(Sigma))
        self.V_t = V_t
        self.b = bias

    def hybrid_forward(self, F, x):
        return F.softmax(x)

In [20]:
U.shape, Sigma.shape, V.shape

((33278, 200), (200,), (200, 200))

In [18]:
# new_decoder = mx.gluon.nn.HybridSequential()
# new_decoder.add(Softmax())
# new_decoder.add(gluon.nn.Dense(33278, in_units=200))
# new_decoder.initialize()
# model.decoder = new_decoder
# model

In [ ]:
evaluate(model, test_data, batch_size, context)

In [15]:
new_decoder = mx.gluon.nn.HybridSequential()
new_decoder.add(mx.gluon.nn.Dense(units=100, activation='relu', flatten=False))
new_decoder.initialize()

model.decoder = new_decoder

In [16]:
model

StandardRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 200, float32)
    (1): Dropout(p = 0.2, axes=())
  )
  (encoder): LSTM(200 -> 200, TNC, num_layers=2, dropout=0.2)
  (decoder): HybridSequential(
    (0): Dense(None -> 100, Activation(relu))
  )
)

In [ ]:
evaluate(model, test_data, batch_size, context)

In [32]:
def train(model, train_data, val_data, test_data, epochs, lr):
    best_val = float("Inf")
    start_train_time = time.time()
    parameters = model.collect_params().values()
    for epoch in range(epochs):
        total_L = 0.0
        start_epoch_time = time.time()
        start_log_interval_time = time.time()
        hiddens = [model.begin_state(batch_size//len(context), func=mx.nd.zeros, ctx=ctx)
                   for ctx in context]
        for i, (data, target) in enumerate(train_data):
            data_list = gluon.utils.split_and_load(data, context,
                                                   batch_axis=1, even_split=True)
            target_list = gluon.utils.split_and_load(target, context,
                                                     batch_axis=1, even_split=True)
            hiddens = detach(hiddens)
            L = 0
            Ls = []
            with autograd.record():
                for j, (X, y, h) in enumerate(zip(data_list, target_list, hiddens)):
                    output, h = model(X, h)
                    batch_L = loss(output.reshape(-3, -1), y.reshape(-1,))
                    L = L + batch_L.as_in_context(context[0]) / (len(context) * X.size)
                    Ls.append(batch_L / (len(context) * X.size))
                    hiddens[j] = h
            L.backward()
            grads = [p.grad(x.context) for p in parameters for x in data_list]
            gluon.utils.clip_global_norm(grads, grad_clip)

            trainer.step(1)

            total_L += sum([mx.nd.sum(l).asscalar() for l in Ls])

            if i % log_interval == 0 and i > 0:
                cur_L = total_L / log_interval
                print('[Epoch %d Batch %d/%d] loss %.2f, ppl %.2f, '
                      'throughput %.2f samples/s'%(
                    epoch, i, len(train_data), cur_L, math.exp(cur_L),
                    batch_size * log_interval / (time.time() - start_log_interval_time)))
                total_L = 0.0
                start_log_interval_time = time.time()

        mx.nd.waitall()

        print('[Epoch %d] throughput %.2f samples/s'%(
                    epoch, len(train_data)*batch_size / (time.time() - start_epoch_time)))
        val_L, val_time = evaluate(model, val_data, batch_size, context[0])
        print('[Epoch %d] time cost %.2fs, valid loss %.2f, valid ppl %.2f'%(
            epoch, time.time()-start_epoch_time, val_L, math.exp(val_L)))

        if val_L < best_val:
            best_val = val_L
            test_L, test_time = evaluate(model, test_data, batch_size, context[0])
            model.save_parameters('{}_{}-{}.params'.format(model_name, dataset_name, epoch))
            print('test loss %.2f, test ppl %.2f, test time %.2f'%(
                test_L, math.exp(test_L), test_time))
        else:
            lr = lr*0.25
            print('Learning rate now %f'%(lr))
            trainer.set_learning_rate(lr)

    print('Total training throughput %.2f samples/s'%(
                            (batch_size * len(train_data) * epochs) /
                            (time.time() - start_train_time)))

In [7]:
train(model, train_data, val_data, test_data, epochs, lr)

[Epoch 0 Batch 200/2983] loss 7.66, ppl 2129.11, throughput 281.65 samples/s
[Epoch 0 Batch 400/2983] loss 6.75, ppl 853.92, throughput 286.11 samples/s
[Epoch 0 Batch 600/2983] loss 6.34, ppl 568.07, throughput 286.57 samples/s
[Epoch 0 Batch 800/2983] loss 6.18, ppl 483.26, throughput 286.67 samples/s
[Epoch 0 Batch 1000/2983] loss 6.05, ppl 422.96, throughput 286.33 samples/s
[Epoch 0 Batch 1200/2983] loss 5.97, ppl 391.82, throughput 286.72 samples/s
[Epoch 0 Batch 1400/2983] loss 5.87, ppl 352.51, throughput 286.17 samples/s
[Epoch 0 Batch 1600/2983] loss 5.86, ppl 351.89, throughput 286.47 samples/s
[Epoch 0 Batch 1800/2983] loss 5.70, ppl 299.67, throughput 286.51 samples/s
[Epoch 0 Batch 2000/2983] loss 5.67, ppl 289.43, throughput 286.45 samples/s
[Epoch 0 Batch 2200/2983] loss 5.57, ppl 261.92, throughput 283.67 samples/s
[Epoch 0 Batch 2400/2983] loss 5.58, ppl 264.64, throughput 287.07 samples/s
[Epoch 0 Batch 2600/2983] loss 5.56, ppl 259.61, throughput 286.79 samples/s
[E